In [41]:
import pandas as pd
dane = pd.read_csv("churn.txt")


In [53]:
dane = dane.replace("False.","False")
dane = dane.replace("True.","True")
dane1 = dane[["Churn?","CustServ Calls"]]
dane1

,Churn?,CustServ Calls
0,False,1
1,False,1
2,False,0
3,False,2
4,False,3
...,...,...
3328,False,2
3329,False,3
3330,False,2
3331,False,2


In [54]:
def fun1(x):
    if x["CustServ Calls"] ==1 or x["CustServ Calls"] == 0 : return "małe"
    if x["CustServ Calls"] ==2 or x["CustServ Calls"] == 3 : return "średnie"
    if x["CustServ Calls"] >3 : return "wysokie"
dane1["CSC"] = dane1.apply(fun1,axis=1)

C:\Users\studentwftims.LAB\AppData\Local\Temp\ipykernel_10388\918173607.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dane1["CSC"] = dane1.apply(fun1,axis=1)


In [55]:
dane1.head()

,Churn?,CustServ Calls,CSC
0,False,1,małe
1,False,1,małe
2,False,0,małe
3,False,2,średnie
4,False,3,średnie


In [56]:
def fun2(x):

    result = pd.Series(index=["małe","średnie","wysokie","suma"],dtype="object")
    result.name="CSC"
    n = len(x)
    result["małe"] = x["CSC"].value_counts()["małe"]
    result["średnie"] = x["CSC"].value_counts()["średnie"]
    result["wysokie"] = x["CSC"].value_counts()["wysokie"]
    result["suma"] = len(x)
    return result


group = dane1.groupby("Churn?")
cross_table = group.apply(fun2)
cross_table.loc["suma"] = cross_table.sum()
cross_table


CSC,małe,średnie,wysokie,suma
Churn?,,,,
False,1664,1057,129,2850
True,214,131,138,483
suma,1878,1188,267,3333


In [61]:
def fun4(x):
    #tworzenie kolumny CSC_sr
    if x["CSC"] =="małe" : return 0
    if x["CSC"] =="średnie" : return 1
    if x["CSC"] =="wysokie" : return 0
def fun5(x):
    #tworzenie parametru CSC_wys
    if x["CSC"] =="małe" : return 0
    if x["CSC"] =="średnie" : return 0
    if x["CSC"] =="wysokie" : return 1

dane1["CSC_sr"] = dane1.apply(fun4,axis=1)
dane1["CSC_wys"] = dane1.apply(fun5,axis=1)

dane1.loc[0:15]

C:\Users\studentwftims.LAB\AppData\Local\Temp\ipykernel_10388\257231657.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dane1["CSC_sr"] = dane1.apply(fun4,axis=1)
C:\Users\studentwftims.LAB\AppData\Local\Temp\ipykernel_10388\257231657.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dane1["CSC_wys"] = dane1.apply(fun5,axis=1)


,Churn?,CustServ Calls,CSC,CSC_sr,CSC_wys
0,False,1,małe,0,0
1,False,1,małe,0,0
2,False,0,małe,0,0
3,False,2,średnie,1,0
4,False,3,średnie,1,0
5,False,0,małe,0,0
6,False,3,średnie,1,0
7,False,0,małe,0,0
8,False,1,małe,0,0
9,False,0,małe,0,0


In [64]:
szansa_male = cross_table["małe"].loc["True"]/cross_table["małe"].loc["False"]
szansa_sr = cross_table["średnie"].loc["True"]/cross_table["średnie"].loc["False"]
szansa_wys = cross_table["wysokie"].loc["True"]/cross_table["wysokie"].loc["False"]

OR_nis = szansa_male
OR_sr = szansa_sr/szansa_male
OR_wys = szansa_wys/szansa_male
print(f"OR_nis={OR_nis}, OR_sr={OR_sr}, OR_wys={OR_wys}")



OR_nis=0.12860576923076922, OR_sr=0.9636866815798549, OR_wys=8.318191697457074


In [65]:
import math

b0 = math.log(OR_nis)
b1 = math.log(OR_sr)
b2 = math.log(OR_wys)
print(f"b0={OR_nis}, b1={b1}, b2={b2}")


b0=0.12860576923076922, b1=-0.03698905630978355, b2=2.118444887154835


In [67]:
import sympy


print(f"Funckja logitowa jest postaci: g(X)=({round(b0,2)})+({round(b1,2)})x+{round(b2,2)}y")
print(f"Prawdopodobieństwo przeniesienia się pi(X):")
X = sympy.Symbol(f'({round(b0,2)})+({round(b1,2)})x+{round(b2,2)}y',real=True)

sympy.exp(X)/(1+sympy.exp(X))

Funckja logitowa jest postaci: g(X)=(-2.05)+(-0.04)x+2.12y
Prawdopodobieństwo przeniesienia się pi(X):


exp((-2.05)+(-0.04)x+2.12y)/(exp((-2.05)+(-0.04)x+2.12y) + 1)

In [68]:
def f3(x,y):
    g = b0 + b1*x + b2*y
    return math.exp(g)/(math.exp(g)+1)

print(f"Prawdopodobieństwo przeniesienia się dla klienta, ktory odbył małą ilosć rozmów: {round(f3(0,0),2)}")
print(f"Prawdopodobieństwo przeniesienia się dla klienta, ktory odbył średnią ilość rozmów: {round(f3(1,0),2)}")
print(f"Prawdopodobieństwo przeniesienia się dla klienta, ktory odbył dużą ilosć rozmów: {round(f3(0,1),2)}")
print(f"Prawdopodobieństwo przeniesienia się klienta ogólnie: {round(cross_table['suma']['True']/cross_table['suma']['suma'],2)}")

Prawdopodobieństwo przeniesienia się dla klienta, ktory odbył małą ilosć rozmów: 0.11
Prawdopodobieństwo przeniesienia się dla klienta, ktory odbył średnią ilość rozmów: 0.11
Prawdopodobieństwo przeniesienia się dla klienta, ktory odbył dużą ilosć rozmów: 0.52
Prawdopodobieństwo przeniesienia się klienta ogólnie: 0.14


In [51]:
print("Wniosek: Odbycie dużej liczby rozmów przez klienta znacząco zwiększa szanse na to, że zmieni on sieć komórkową.")

wniosek: Odbycie dużej liczby rozmów przez klienta znacząco zwiększa szanse na to, że zmieni on sieć komórkową.
